In [1]:
import requests
from bs4 import BeautifulSoup
import time
import sys

#cookie資料
payload = {
    "from" : "/bbs/Beauty/M.1546412589.A.A27.html",
    "yes" : "yes"
}
rs = requests.session() #save information in one cookie
rs.post("https://www.ptt.cc/ask/over18", data = payload)

<Response [200]>

In [57]:
if (True):
    file_all = open("all_articles.txt", "w", encoding="utf-8")
    file_popular = open("all_popular.txt", "w", encoding="utf-8")

    for i in range(2748, 3144):
        #防止被鎖IP
        time.sleep(0.01)
        #2019的頁數選擇
        url = "https://www.ptt.cc/bbs/Beauty/index" + str(i) + ".html"
        res = rs.get(url)
        content = res.text
        soup = BeautifulSoup(content, "html.parser")
        #print(soup.text)
        title_list = soup.select("div.r-ent")
        #print(title_list)
        plain = "https://www.ptt.cc"

        for item in title_list:
            if (item.select_one("div.title").select_one("a") == None):
                continue

            #找日期
            item_date = item.select_one("div.date").text
            if (i == 2748 and item_date[0 : item_date.find("/")] == "12"):
                continue
            elif (i == 3143 and item_date[1 : item_date.find("/")] == "1"):
                continue
            else:
                if (item_date[0] == " "):
                    item_date = item_date[1 : item_date.find("/")] + item_date[item_date.find("/")+1 : ]
                else:
                    item_date = item_date[0 : item_date.find("/")] + item_date[item_date.find("/")+1 : ]
            #print(item_date)

            #找標題
            item_title = item.select_one("div.title").select_one("a").text
            #print(item_title)

            #找網址
            item_href = item.select_one("div.title").select_one("a").get("href")
            item_href = plain + item_href
            #print(item_href)

            #檔案內容格式
            result = item_date + "," + item_title + "," + item_href

            #判斷是否爆文
            if (item.select_one("div.nrec").text == "爆"):
                #print(result)
                file_popular.write(result + "\n")

            #所有2019文章(2748~3143)
            if item_title.find("[公告]") == -1:
                #print(result)
                file_all.write(result + "\n")        
    file_all.close()
    file_popular.close()

In [55]:
start_date, end_date = "420", "426"
file_all = open("all_articles.txt", "r", encoding="utf-8")
file_push = open("push.txt", "w", encoding="utf-8")
article = file_all.readlines()
#print(article)
#print(article[0])

count_hot, count_cold = 0, 0
like_list, boo_list = [], []
for item in article:
    time.sleep(0.01)
    #需扣掉換行字元\n
    url = item[item.find("https") : len(item)-1]
    #時間>end_date直接跳出迴圈
    if (int(item[0 : item.find(",")]) > int(end_date)):
        break
    #時間<start_date直接下一輪迴圈
    if (int(item[0 : item.find(",")]) < int(start_date)):
        continue
    res = rs.get(url)
    content = res.text
    soup = BeautifulSoup(content, "html.parser")
    #找出每一則留言
    push_title = soup.select("div.push")
    #print(push_title)
    #統計留言裡是推文或噓文
    for member in push_title:
        span_list = member.select("span")
        #留言過多的例外 -> len(span_list) == 0
        if (len(span_list) == 0):
            continue
        print(span_list)
        print("\n\n")
        #print(span_list[0].text)
        if (span_list[0].text == "推 "):
            count_hot += 1
            #空list
            if (len(like_list) == 0):
                like_list.append([span_list[1].text, 1])
            #非空list
            else:
                for i in range(len(like_list)):
                    #此人之前出現過
                    if (span_list[1].text in like_list[i]):
                        like_list[i][1] += 1
                        break
                    #此人之前未出現過
                    if (i == len(like_list)-1):
                        like_list.append([span_list[1].text, 1])
        if (span_list[0].text == "噓 "):
            count_cold += 1
            #空list
            if (len(boo_list) == 0):
                boo_list.append([span_list[1].text, 1])
            #非空list
            else:
                for i in range(len(boo_list)):
                    #此人之前出現過
                    if (span_list[1].text in boo_list[i]):
                        boo_list[i][1] += 1
                        break
                    #此人之前未出現過
                    if (i == len(boo_list)-1):
                        boo_list.append([span_list[1].text, 1])
'''                    
like_count = 0
boo_count = 0
for i in range(len(like_list)):
    like_count += like_list[i][1]
for i in range(len(boo_list)):
    boo_count += boo_list[i][1]
print(like_count)
print(boo_count)'''

like_list.sort(key = lambda score: score[0])
like_list.sort(key = lambda score: score[1], reverse = True)
boo_list.sort(key = lambda score: score[0])
boo_list.sort(key = lambda score: score[1], reverse = True)

'''
print("all like: " + str(count_hot))
print("all boo: " + str(count_cold))
print(like_list)
print(boo_list)'''

#輸出資料到file_push.txt
file_push.write("all like: " + str(count_hot) + "\n" + "all boo: " + str(count_cold) + "\n")
for i in range(10):
    file_push.write("like #" + str(i+1) + ": " + like_list[i][0] + " " + str(like_list[i][1]) + "\n")
for i in range(10):
    file_push.write("boo #" + str(i+1) + ": " + boo_list[i][0] + " " + str(boo_list[i][1]) + "\n")    
       
file_all.close()
file_push.close()

[<span class="f1 hl push-tag">噓 </span>, <span class="f3 hl push-userid">Jmayu</span>, <span class="f3 push-content">: 左轉yt板</span>, <span class="push-ipdatetime"> 04/20 01:50
</span>]



[<span class="hl push-tag">推 </span>, <span class="f3 hl push-userid">zxzxzzzx</span>, <span class="f3 push-content">: 肥宅還沒鬧夠啊，桶粉果然一群智障</span>, <span class="push-ipdatetime"> 04/20 02:09
</span>]



[<span class="f1 hl push-tag">噓 </span>, <span class="f3 hl push-userid">jimmy51309</span>, <span class="f3 push-content">: 請洽專版</span>, <span class="push-ipdatetime"> 04/20 02:32
</span>]



[<span class="f1 hl push-tag">噓 </span>, <span class="f3 hl push-userid">jackie1115</span>, <span class="f3 push-content">: 鬧板，滾</span>, <span class="push-ipdatetime"> 04/20 02:46
</span>]



[<span class="f1 hl push-tag">噓 </span>, <span class="f3 hl push-userid">sanxiao</span>, <span class="f3 push-content">: 幹嘛幹嘛</span>, <span class="push-ipdatetime"> 04/20 10:00
</span>]



[<span class="hl push-tag">推 </span>, <s

[<span class="hl push-tag">推 </span>, <span class="f3 hl push-userid">belucky</span>, <span class="f3 push-content">: 版皇不下台的    不然怎麼當版皇</span>, <span class="push-ipdatetime"> 04/20 05:17
</span>]



[<span class="hl push-tag">推 </span>, <span class="f3 hl push-userid">wzshi</span>, <span class="f3 push-content">: 版主下台 還表特寧靜</span>, <span class="push-ipdatetime"> 04/20 05:17
</span>]



[<span class="hl push-tag">推 </span>, <span class="f3 hl push-userid">ClownT</span>, <span class="f3 push-content">: 應該打算等風頭過吧 看準大家鬧也頂多一兩個禮拜</span>, <span class="push-ipdatetime"> 04/20 05:18
</span>]



[<span class="hl push-tag">推 </span>, <span class="f3 hl push-userid">Archerdark</span>, <span class="f3 push-content">: 按照政客標準sop 打算裝死一陣子等風頭過吧</span>, <span class="push-ipdatetime"> 04/20 05:32
</span>]



[<span class="hl push-tag">推 </span>, <span class="f3 hl push-userid">az564232</span>, <span class="f3 push-content">: 一躲天下無難事啊</span>, <span class="push-ipdatetime"> 04/20 05:34
</span>]



[<span cl

</span>]



[<span class="hl push-tag">推 </span>, <span class="f3 hl push-userid">abcte</span>, <span class="f3 push-content">: 我是覺得 板主乾脆霸氣一點 宣布表特版個版化 並且公開板主</span>, <span class="push-ipdatetime"> 04/20 09:34
</span>]



[<span class="hl push-tag">推 </span>, <span class="f3 hl push-userid">aduijjr</span>, <span class="f3 push-content">: 表皇要上朝了嗎</span>, <span class="push-ipdatetime"> 04/20 09:34
</span>]



[<span class="f1 hl push-tag">→ </span>, <span class="f3 hl push-userid">abcte</span>, <span class="f3 push-content">: 認可的帥哥美女清單 不在清單內的一律桶 這樣很公正很公平</span>, <span class="push-ipdatetime"> 04/20 09:35
</span>]



[<span class="hl push-tag">推 </span>, <span class="f3 hl push-userid">kuan50118</span>, <span class="f3 push-content">: 親友團還真多呢</span>, <span class="push-ipdatetime"> 04/20 09:35
</span>]



[<span class="hl push-tag">推 </span>, <span class="f3 hl push-userid">taiwan540</span>, <span class="f3 push-content">: 對啊 直接宣布個版化吧，反正也差不多了</span>, <span class="push-ipdatetime"> 04/20 09:

</span>]



[<span class="f1 hl push-tag">噓 </span>, <span class="f3 hl push-userid">rhl21</span>, <span class="f3 push-content">: 我只看到一堆沙雕在亂</span>, <span class="push-ipdatetime"> 04/20 12:35
</span>]



[<span class="f1 hl push-tag">→ </span>, <span class="f3 hl push-userid">iamdead5566</span>, <span class="f3 push-content">: 你現在覺得亂板很煩 代表就是有效了好嗎...現在是要</span>, <span class="push-ipdatetime"> 04/20 12:35
</span>]



[<span class="f1 hl push-tag">→ </span>, <span class="f3 hl push-userid">kakain</span>, <span class="f3 push-content">: 畢竟如果板主把他禁止的原因解釋清楚 再舉辦投票 我覺得新</span>, <span class="push-ipdatetime"> 04/20 12:36
</span>]



[<span class="f1 hl push-tag">→ </span>, <span class="f3 hl push-userid">iamdead5566</span>, <span class="f3 push-content">: 版皇出來不要裝死</span>, <span class="push-ipdatetime"> 04/20 12:36
</span>]



[<span class="f1 hl push-tag">→ </span>, <span class="f3 hl push-userid">kakain</span>, <span class="f3 push-content">: 版規應該會過 也不至於鬧到現在這麼難看</span>, <span class="push-ipdat

[<span class="hl push-tag">推 </span>, <span class="f3 hl push-userid">ggggg50713</span>, <span class="f3 push-content">: 因該/而以/不準去/沒關西/說的好/在來一次/從來一遍</span>, <span class="push-ipdatetime">12/20 02:01
</span>]



[<span class="f1 hl push-tag">噓 </span>, <span class="f3 hl push-userid">morgen</span>, <span class="f3 push-content">: 他是板主,不爽他可以去罷免他,鬧版是傻逼屁孩行徑,該桶!</span>, <span class="push-ipdatetime">04/20 09:26
</span>]



[<span class="hl push-tag">推 </span>, <span class="f3 hl push-userid">rsgarlic</span>, <span class="f3 push-content">: 板皇粉就腦袋轉不過來 跟他們解釋這麼多也沒用</span>, <span class="push-ipdatetime">04/20 09:26
</span>]



[<span class="hl push-tag">推 </span>, <span class="f3 hl push-userid">talk5566</span>, <span class="f3 push-content">: 應該是回到蔣介石的白色恐怖 跟對岸的不同</span>, <span class="push-ipdatetime">04/20 09:27
</span>]



[<span class="f1 hl push-tag">→ </span>, <span class="f3 hl push-userid">greg7575</span>, <span class="f3 push-content">: 這叫綠共，小朋友別學</span>, <span class="push-ipdatetime">0

...

[<span class="f1 hl push-tag">→ </span>, <span class="f3 hl push-userid">william12tw</span>, <span class="f3 push-content">: 沒了冷凍迪 沒啥好看的</span>, <span class="push-ipdatetime"> 04/26 22:47
</span>]



[<span class="f1 hl push-tag">→ </span>, <span class="f3 hl push-userid">selfhu</span>, <span class="f3 push-content">: 2壞了</span>, <span class="push-ipdatetime"> 04/26 22:47
</span>]



[<span class="hl push-tag">推 </span>, <span class="f3 hl push-userid">darkhearo</span>, <span class="f3 push-content">: 品皇不下臺 表特不會好，對不起我打錯了...</span>, <span class="push-ipdatetime"> 04/26 22:48
</span>]



[<span class="f1 hl push-tag">→ </span>, <span class="f3 hl push-userid">Forthelife</span>, <span class="f3 push-content">: 2壞了，中國人覺得正常</span>, <span class="push-ipdatetime"> 04/26 22:48
</span>]



[<span class="f1 hl push-tag">→ </span>, <span class="f3 hl push-userid">sister47</span>, <span class="f3 push-content">: 最正的就幫主小迪 唉...</span>, <span class="push-ipdatetime"> 04/26 22:57
</span>]



[<span cl

</span>]



[<span class="hl push-tag">推 </span>, <span class="f3 hl push-userid">Ken1843</span>, <span class="f3 push-content">: 版皇下台</span>, <span class="push-ipdatetime"> 04/26 23:34
</span>]



[<span class="hl push-tag">推 </span>, <span class="f3 hl push-userid">ianchen1223</span>, <span class="f3 push-content">: 版皇下台</span>, <span class="push-ipdatetime"> 04/26 23:35
</span>]



[<span class="hl push-tag">推 </span>, <span class="f3 hl push-userid">wwjd198323</span>, <span class="f3 push-content">: 推</span>, <span class="push-ipdatetime"> 04/26 23:35
</span>]



[<span class="hl push-tag">推 </span>, <span class="f3 hl push-userid">nonis</span>, <span class="f3 push-content">: 下台</span>, <span class="push-ipdatetime"> 04/26 23:36
</span>]



[<span class="hl push-tag">推 </span>, <span class="f3 hl push-userid">kingcatkuo</span>, <span class="f3 push-content">: 版皇下台</span>, <span class="push-ipdatetime"> 04/26 23:36
</span>]



[<span class="f1 hl push-tag">→ </span>, <span class="f

</span>]



[<span class="hl push-tag">推 </span>, <span class="f3 hl push-userid">tsung0823</span>, <span class="f3 push-content">: 推</span>, <span class="push-ipdatetime"> 04/27 00:14
</span>]



[<span class="hl push-tag">推 </span>, <span class="f3 hl push-userid">cwh0105</span>, <span class="f3 push-content">: 結衣 我婆</span>, <span class="push-ipdatetime"> 04/27 00:14
</span>]



[<span class="hl push-tag">推 </span>, <span class="f3 hl push-userid">jajatje</span>, <span class="f3 push-content">: 板皇下台</span>, <span class="push-ipdatetime"> 04/27 00:14
</span>]



[<span class="hl push-tag">推 </span>, <span class="f3 hl push-userid">aquariuschi</span>, <span class="f3 push-content">: 版皇下台</span>, <span class="push-ipdatetime"> 04/27 00:15
</span>]



[<span class="hl push-tag">推 </span>, <span class="f3 hl push-userid">hahahui</span>, <span class="f3 push-content">: 版皇下台</span>, <span class="push-ipdatetime"> 04/27 00:15
</span>]



[<span class="hl push-tag">推 </span>, <span class="f3 

[<span class="hl push-tag">推 </span>, <span class="f3 hl push-userid">Bigwastrel</span>, <span class="f3 push-content">: 推</span>, <span class="push-ipdatetime"> 04/27 15:28
</span>]



[<span class="hl push-tag">推 </span>, <span class="f3 hl push-userid">qq7769912</span>, <span class="f3 push-content">: 版皇下台</span>, <span class="push-ipdatetime"> 04/27 16:59
</span>]



[<span class="hl push-tag">推 </span>, <span class="f3 hl push-userid">ken0103010</span>, <span class="f3 push-content">: 版皇下台</span>, <span class="push-ipdatetime"> 04/27 18:29
</span>]



[<span class="hl push-tag">推 </span>, <span class="f3 hl push-userid">RichardTu</span>, <span class="f3 push-content">: 板皇下台</span>, <span class="push-ipdatetime"> 04/27 18:52
</span>]



[<span class="f1 hl push-tag">→ </span>, <span class="f3 hl push-userid">laytomall</span>, <span class="f3 push-content">: 沒航航好看 下去</span>, <span class="push-ipdatetime"> 04/28 00:17
</span>]



[<span class="hl push-tag">推 </span>, <span class="f3 

[<span class="hl push-tag">推 </span>, <span class="f3 hl push-userid">gash9890</span>, <span class="f3 push-content">: 有更適台的板</span>, <span class="push-ipdatetime"> 04/26 23:46
</span>]



[<span class="f1 hl push-tag">→ </span>, <span class="f3 hl push-userid">darkhearo</span>, <span class="f3 push-content">: 樓上勇者</span>, <span class="push-ipdatetime"> 04/26 23:46
</span>]



[<span class="hl push-tag">推 </span>, <span class="f3 hl push-userid">lonelyboy</span>, <span class="f3 push-content">: 致敬勇者</span>, <span class="push-ipdatetime"> 04/26 23:47
</span>]



[<span class="hl push-tag">推 </span>, <span class="f3 hl push-userid">flame1030</span>, <span class="f3 push-content">: ....</span>, <span class="push-ipdatetime"> 04/26 23:48
</span>]



[<span class="hl push-tag">推 </span>, <span class="f3 hl push-userid">tiaushiwan</span>, <span class="f3 push-content">: 我婆可愛</span>, <span class="push-ipdatetime"> 04/26 23:50
</span>]



[<span class="f1 hl push-tag">→ </span>, <span class="f

In [58]:
start_date, end_date = "420", "426"
file_popular = open("all_popular.txt", "r", encoding="utf-8")
file_image = open("popular.txt", "w", encoding="utf-8")
article = file_popular.readlines()
#print(article)

popular_count = 0
output_list = []
for item in article:
    time.sleep(0.01)
    #需扣掉換行字元\n
    url = item[item.find("https") : len(item)-1]
    #時間>end_date直接跳出迴圈
    if (int(item[0 : item.find(",")]) > int(end_date)):
        break
    #時間<start_date直接下一輪迴圈
    if (int(item[0 : item.find(",")]) < int(start_date)):
        continue
    popular_count += 1
    
    res = rs.get(url)
    content = res.text
    soup = BeautifulSoup(content, "html.parser")
    href_list = soup.select("a")
    
    image_count = 0
    for item in href_list:
        image_url = item.get("href")
        if (image_url[-3:].lower() == "jpg" or image_url[-4:].lower() == "jpeg" or image_url[-3:].lower() == "png" or image_url[-3:].lower() == "gif"):
            image_count += 1
            output_list.append(image_url)
            #print(image_url)
    #print(len(output_list))        
    #print(image_count)    
#print(popular_count)
  
file_image.write("number of popular articles: " + str(popular_count) + "\n")
for item in output_list:
    file_image.write(item + "\n")
    
file_popular.close()
file_image.close()

In [19]:
keyword = "GeminMan"
start_date, end_date = "420", "421"
file_all = open("all_articles.txt", "r", encoding="utf-8")
file_image = open("keyword.txt", "w", encoding="utf-8")
article = file_all.readlines()

output_list = []
for item in article:
    time.sleep(0.01)
    #需扣掉換行字元\n
    url = item[item.find("https") : len(item)-1]
    #時間>end_date直接跳出迴圈
    if (int(item[0 : item.find(",")]) > int(end_date)):
        break
    #時間<start_date直接下一輪迴圈
    if (int(item[0 : item.find(",")]) < int(start_date)):
        continue
    
    res = rs.get(url)
    content = res.text
    soup = BeautifulSoup(content, "html.parser")
    href_list = soup.select("a")
    article = soup.text

    #確認是否符合發信帖格式
    if(article.find("※ 發信站") > 0):
        article = article[article.find("作者") : article.find("※ 發信站")]
        #確認是否有keyword
        if (article.find(keyword) > 0):
            #image_count = 0
            #找圖片url
            for item in href_list:
                image_url = item.get("href")
                if (image_url[-3:].lower() == "jpg" or image_url[-4:].lower() == "jpeg" or image_url[-3:].lower() == "png" or image_url[-3:].lower() == "gif"):
                    #image_count += 1
                    output_list.append(image_url)
            #print(image_count)

for item in output_list:
    file_image.write(item + "\n")
    
file_all.close()
file_image.close()    

36
67


In [43]:
url = "https://www.ptt.cc/bbs/Beauty/M.1555708355.A.0F3.html"
res = rs.get(url)
content = res.text
soup = BeautifulSoup(content, "html.parser")

href_list = soup.select("a")
for item in href_list:
    image_url = item.get("href")
    if (image_url.find(".jpg") > 0 or image_url.find(".jpeg") > 0 or image_url.find(".png") > 0 or image_url.find(".gif") > 0):
        print(image_url)

https://i.imgur.com/LT2brUo.jpg
https://i.imgur.com/JxDkxbZ.jpg
https://i.imgur.com/SneDq1N.jpg
https://i.imgur.com/F91qerq.jpg
https://i.imgur.com/yuLZQmH.jpg
https://i.imgur.com/sTWiwU7.jpg
https://i.imgur.com/GTOIbdf.jpg
https://i.imgur.com/vlmOCO1.jpg
https://i.imgur.com/KXGoCv4.jpg
https://i.imgur.com/vgwWHUg.jpg
https://i.imgur.com/QO0eZe9.jpg
https://i.imgur.com/QV7FnDq.jpg
https://i.imgur.com/UWAF8R7.jpg
https://i.imgur.com/7fSmRdz.jpg
https://i.imgur.com/Qiu7NZI.jpg
https://i.imgur.com/KnxHPr0.jpg
https://i.imgur.com/Wf82u8Z.jpg
https://i.imgur.com/t54gztF.jpg
https://i.imgur.com/YQXTCEq.jpg
https://i.imgur.com/nzHiFcc.jpg
https://i.imgur.com/S0B4o3X.jpg
https://i.imgur.com/XViC87x.jpg
https://i.imgur.com/FX7YLV3.jpg
https://i.imgur.com/xRMlO6c.jpg
https://i.imgur.com/d0e4uQL.jpg
https://i.imgur.com/F1ET0Ao.jpg
https://i.imgur.com/s4U70Lx.jpg
https://i.imgur.com/XXdLeeA.jpg
https://i.imgur.com/12MUrVl.jpg
https://i.imgur.com/qK5wAMw.jpg
https://i.imgur.com/T2OTLdU.jpg
https://

In [7]:
keyword = "看板"

url = "https://www.ptt.cc/bbs/Beauty/M.1555708355.A.0F3.html"
res = rs.get(url)
content = res.text
soup = BeautifulSoup(content, "html.parser")
article = soup.text

if(article.find("※ 發信站") > 0):
    article = article[article.find("作者") : article.find("※ 發信站")]
    print(article)

if (article.find(keyword) > 0):
    print("hi")

作者tpc302 (撐過就是你的)看板Beauty標題[討論] 板主GeminiMan請問您何時要請辭下臺負責?時間Sat Apr 20 05:12:33 2019
表特板之亂已經有幾天了

亂源應該是板主GeminMan您發的公告 （有誤請指正

提到統神文（張家兄弟) 應有更合適的看板可供張貼開始

等於直接禁貼統神文 請問統神何錯之有?

其他各大板如NBA板要禁貼LYS文 棒球板要禁某記者文

也是經過投票該改為限一日一文

再改到現在全面禁貼

都是經過板友投票 取得共識通過的板規

現在您說要禁就禁 不就如同個人獨裁決定板規

臺灣是個民主國家社會 並非共產獨裁專政

民主早已落實於社會各層面

更遑論ptt

這個政策沒有取得板友共識同意

一意孤行就實施

引起板友不滿 造成板友以表特板之亂向您抗議

而您卻以各種水桶回應

罪刑應法定 但此法應是照正常程序通過的法 才符合程序正義

不然水桶也是言不正名不順

而欲解決表特板之亂的辦法竟是調高發言門檻???

請問表特板是NBA打季後賽了還是要戒嚴?

事實證明您已不適任表特板主

一個大板被您管成這樣

越管越亂

還不知道亂源及問題所在

倒不如無為而治

您應該知所進退 主動請辭下臺負責止血吧

不然水桶再多人

只是讓表特板越來越沒人來而已

表特板曾是個美好的地方

帶給人們快樂及釋放壓力

讓人們看見世界的美好

而不是現在這樣……

夠了吧

可以讓表特板恢復正常了吧?

Make Beauty Great Again

謝謝各位板友看完

個人拙見請板友參與討論

謝謝大家

https://i.imgur.com/LT2brUo.jpg

https://i.imgur.com/JxDkxbZ.jpg

https://i.imgur.com/SneDq1N.jpg

https://i.imgur.com/F91qerq.jpg

https://i.imgur.com/yuLZQmH.jpg

https://i.imgur.com/sTWiwU7.jpg

https://i.imgur.com/GTOIbdf.jpg

https://i.imgur.com/vlmOCO1.jpg

https://i.imgur.com/KXGoCv4.jpg

https://i.im

In [59]:
list = [[1, 2], [3, 4]]
print([1, 2] not in list)
print(list[0].index(2))

list = [["apple", 4], ["Apple", 2]]
print(list)
list.sort(key = lambda score: score[0])
print(list)

False
1
[['apple', 4], ['Apple', 2]]
[['Apple', 2], ['apple', 4]]
